In [1]:
import os
import math
import datetime
from mlsettings.settings import load_app_config, get_datafolder_path
from mltools.mlcommon import (load_data, print_dataset_info, split_dataset, 
                              auto_scatter_simple,load_dataset,detect_outliers,
                              one_hot_dataframe)

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline 
from numpy import set_printoptions
set_printoptions(precision=4)

pd.set_option('display.width', 140)
pd.set_option('precision', 4)

sns.set_style("whitegrid")

sns.set_palette(sns.color_palette("muted", 15))

In [2]:
import joblib

In [3]:
catboost_data = pd.read_csv('titanic_test_catboost.csv')

In [4]:
catboost_data.head()

,Age,Embarked,Fare,Parch,Pclass,Sex,SibSp,Title,Deck_Level
0,34.5,Q,7.8292,0,3,male,0,Mr,X
1,47.0,S,7.0000,0,3,female,1,Mrs,X
2,62.0,Q,9.6875,0,2,male,0,Mr,X
3,27.0,S,8.6625,0,3,male,0,Mr,X
4,22.0,S,12.2875,1,3,female,1,Mrs,X


In [5]:
catboost_clf = joblib.load('catboost.jbl')

In [6]:
y_cb = catboost_clf.predict(catboost_data)

In [7]:
xgboost_data =pd.read_csv('titanic_test_xbgoost.csv')

In [8]:
xgboost_clf = joblib.load('xgboost.jbl')

In [9]:
import xgboost as xgb
xgboost_data =xgb.DMatrix(xgboost_data)

In [10]:
y_xgb = xgboost_clf.predict(xgboost_data)

In [11]:
pred_df  =pd.DataFrame(list(zip(y_cb,y_xgb)) ,columns =['CatBoost','XGBoost'])

In [12]:
pred_df[pred_df['CatBoost']== pred_df['XGBoost']]

,CatBoost,XGBoost
0,0.0,0.0
2,0.0,0.0
3,0.0,0.0
5,0.0,0.0
6,1.0,1.0
...,...,...
413,0.0,0.0
414,1.0,1.0
415,0.0,0.0
416,0.0,0.0


In [13]:
differing = pred_df[pred_df['CatBoost']!= pred_df['XGBoost']]

In [14]:
differing.shape

(23, 2)

In [62]:
y_cb_proba = catboost_clf.predict_proba(catboost_data)

In [75]:
y_cb_proba[:10]

array([[0.8524, 0.1476],
       [0.4713, 0.5287],
       [0.8144, 0.1856],
       [0.7527, 0.2473],
       [0.5979, 0.4021],
       [0.8295, 0.1705],
       [0.2451, 0.7549],
       [0.873 , 0.127 ],
       [0.1449, 0.8551],
       [0.9345, 0.0655]])

In [91]:
y_cb_2 =[ 1 if probs[1] > 0.55 else 0 for probs in y_cb_proba]
pred_df['catboost_proba'] =y_cb_2

In [92]:
pred_df[pred_df['CatBoost']== pred_df['catboost_proba']]

,CatBoost,XGBoost,catboost_proba
0,0.0,0.0,0
2,0.0,0.0,0
3,0.0,0.0,0
4,0.0,1.0,0
5,0.0,0.0,0
...,...,...,...
413,0.0,0.0,0
414,1.0,1.0,1
415,0.0,0.0,0
416,0.0,0.0,0


In [93]:
pred_df['catboost_proba'].to_csv('proba.csv')